<a href="https://colab.research.google.com/github/NicoleMeinie/Regression/blob/master/Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

# should these be here or in the body of the code? - PEP8
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# avoid pd warnings 
import warnings
warnings.filterwarnings("ignore")

# upload data
Test = pd.read_csv('Test.csv')
Train = pd.read_csv('Train.csv')
Riders = pd.read_csv('Riders.csv')
#Sub = pd.read_csv('SampleSubmission.csv')

# merge drivers's informations with the train and test datasets
Train = Train.merge(Riders, on='Rider Id', how='left')
Test = Test.merge(Riders, on='Rider Id', how='left')

Tests = Test.copy()

In [120]:
from sklearn.impute import SimpleImputer

# Replacing Null values in the temperature column with the average value - train and test set
imputer_temp = SimpleImputer(missing_values = np.NaN, strategy = 'mean')
imputer_temp_1 = SimpleImputer(missing_values = np.NaN, strategy = 'mean')

imputer_temp.fit(Train.iloc[:,20:22])
Train.iloc[:,20:22] = imputer_temp.transform(Train.iloc[:,20:22])

imputer_temp_1.fit(Train.iloc[:,17:19])
Test.iloc[:,17:19] = imputer_temp_1.transform(Test.iloc[:,17:19])

# Replacing the Null values in the precipitation column with 0 - train and test set 
imputer_prec = SimpleImputer(missing_values = np.NaN, strategy = 'constant', fill_value = 0)
imputer_prec_1 = SimpleImputer(missing_values = np.NaN, strategy = 'constant', fill_value = 0)

imputer_prec.fit(Train.iloc[:,22:])
Train.iloc[:,22:] = imputer_prec.transform(Train.iloc[:,22:])

imputer_prec_1.fit(Test.iloc[:,19:22])
Test.iloc[:,19:22] = imputer_prec_1.transform(Test.iloc[:,19:22])  
Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 33 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  int64  
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

In [0]:
# Drop the ID and Vehicle type columns in the train and test data
to_drop = ['Order No', 'User Id', 'Rider Id', 'Vehicle Type', 'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)', 'Confirmation - Time', 
           'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time', 'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)',
             'No_of_Ratings']
Train.drop(to_drop, axis = 1, inplace = True)
Test.drop(to_drop, axis = 1, inplace = True)

In [122]:
# rename some columns
name = {'Placement - Day of Month': 'Day of Month', 'Placement - Weekday (Mo = 1)': 'Weekday'}
Train.rename(columns = name, inplace = True) 
Test.rename(columns = name, inplace = True)
Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 20 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Platform Type                              21201 non-null  int64  
 1   Personal or Business                       21201 non-null  object 
 2   Day of Month                               21201 non-null  int64  
 3   Weekday                                    21201 non-null  int64  
 4   Placement - Time                           21201 non-null  object 
 5   Pickup - Time                              21201 non-null  object 
 6   Arrival at Destination - Day of Month      21201 non-null  int64  
 7   Arrival at Destination - Weekday (Mo = 1)  21201 non-null  int64  
 8   Arrival at Destination - Time              21201 non-null  object 
 9   Distance (KM)                              21201 non-null  float64
 10  Temperature           

In [0]:
# convert Times to timestamps

# Train data
Train['Placement - Time'] = pd.to_datetime(Train['Placement - Time'])
Train['Pickup - Time'] = pd.to_datetime(Train['Pickup - Time'])

# Test data
Test['Placement - Time'] = pd.to_datetime(Test['Placement - Time'])
Test['Pickup - Time'] = pd.to_datetime(Test['Pickup - Time'])

In [0]:
# Create new feature: Waiting time between the time an order was placed and when the order was picked up
Train['Waiting time'] = Train['Pickup - Time'] - Train['Placement - Time']
Test['Waiting time'] = Test['Pickup - Time'] - Test['Placement - Time']

# convert waiting time from time_delta to seconds 
Train['Waiting time'] = Train['Waiting time'].astype('timedelta64[s]')
Test['Waiting time'] = Test['Waiting time'].astype('timedelta64[s]') 

In [126]:
Train.drop(['Placement - Time'], axis = 1, inplace = True)
Test.drop(['Placement - Time'], axis = 1, inplace = True)

Train['Pickup - Time'] = Train['Pickup - Time'].dt.hour
Test['Pickup - Time'] = Test['Pickup - Time'].dt.hour
Test.head()

,Platform Type,Personal or Business,Day of Month,Weekday,Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,Waiting time
0,3,Business,27,3,17,8.0,3.240225,0.0,-1.333275,36.870815,-1.305249,36.822390,439,1511,13.3,1357.0
1,3,Business,17,5,13,5.0,3.240225,0.0,-1.272639,36.794723,-1.277007,36.823907,488,273,14.4,1682.0
2,3,Business,27,4,11,5.0,22.800000,0.0,-1.290894,36.822971,-1.276574,36.851365,833,460,13.1,2980.0
3,3,Business,17,1,14,5.0,24.500000,0.0,-1.290503,36.809646,-1.303382,36.790658,487,560,13.7,1517.0
4,3,Business,11,2,11,6.0,24.400000,0.0,-1.281081,36.814423,-1.266467,36.792161,4761,1120,14.1,1536.0


In [127]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 20 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Platform Type                              21201 non-null  int64  
 1   Personal or Business                       21201 non-null  object 
 2   Day of Month                               21201 non-null  int64  
 3   Weekday                                    21201 non-null  int64  
 4   Pickup - Time                              21201 non-null  int64  
 5   Arrival at Destination - Day of Month      21201 non-null  int64  
 6   Arrival at Destination - Weekday (Mo = 1)  21201 non-null  int64  
 7   Arrival at Destination - Time              21201 non-null  object 
 8   Distance (KM)                              21201 non-null  float64
 9   Temperature                                21201 non-null  float64
 10  Precipitation in milli

In [128]:
# encode personal/business
x_drop = ['Arrival at Destination - Time', 'Arrival at Destination - Weekday (Mo = 1)', 'Arrival at Destination - Day of Month']
Train.drop(x_drop, axis = 1, inplace = True)

# create dummy variables for the train and test data
Train = pd.get_dummies(Train, drop_first=True)
Test = pd.get_dummies(Test, drop_first=True)

Train.head()

,Platform Type,Day of Month,Weekday,Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,Waiting time,Personal or Business_Personal
0,3,9,5,10,4.0,20.400000,0.0,-1.317755,36.830370,-1.300406,36.829741,745,1637,1309,13.8,3104.0,0
1,3,12,5,11,16.0,26.400000,0.0,-1.351453,36.899315,-1.295004,36.814358,1993,396,339,13.6,1673.0,1
2,3,30,2,12,3.0,23.258889,0.0,-1.308284,36.843419,-1.300921,36.828195,455,1023,242,12.5,818.0,0
3,3,15,5,9,9.0,19.200000,0.0,-1.281301,36.832396,-1.257147,36.795063,1341,886,283,14.5,1052.0,0
4,1,13,1,10,9.0,15.400000,0.0,-1.266597,36.792118,-1.295041,36.809817,1214,2311,872,14.1,605.0,1


In [0]:
Train.drop(['Personal or Business_Personal', 'Platform Type', 'Precipitation in millimeters', 'Day of Month'], axis = 1, inplace = True)
Test.drop(['Personal or Business_Personal', 'Platform Type', 'Precipitation in millimeters', 'Day of Month'], axis = 1, inplace = True)

In [0]:
# Scaling data
from sklearn.preprocessing import StandardScaler
Train_s = Train.copy()
Test_s = Test.copy()
col_names = ['Distance (KM)', 'Temperature', 'Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long', 'No_Of_Orders', 'Age', 'Average_Rating', 'Waiting time']

# Train data
train_features = Train_s[col_names]
train_scaler = StandardScaler().fit(train_features.values)
train_features = train_scaler.transform(train_features.values)
Train_s[col_names] = train_features

#Test data
test_features = Test_s[col_names]
test_features = train_scaler.transform(test_features.values)
Test_s[col_names] = test_features


In [0]:
# split train data into predictors and response variables 

X = Train_s.copy()
x_drops = ['Time from Pickup to Arrival']
X.drop(x_drops, inplace = True, axis = 1)

y = Train['Time from Pickup to Arrival']

In [132]:
X.head()


,Weekday,Pickup - Time,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,Waiting time
0,5,10,-0.971370,-8.873220e-01,-1.189424,0.509897,-0.511888,0.414159,-0.035206,0.501451,-0.090082,0.792547
1,5,11,1.145469,9.749159e-01,-2.294056,2.349813,-0.356757,0.070177,-0.823507,-0.998616,-0.308411,-0.193605
2,2,12,-1.147774,1.102666e-15,-0.878981,0.858136,-0.526672,0.379588,-0.425228,-1.148623,-1.509220,-0.782815
3,5,9,-0.089354,-1.259770e+00,0.005523,0.563962,0.730374,-0.361289,-0.512252,-1.085218,0.674069,-0.621558
4,1,10,-0.089354,-2.439187e+00,0.487515,-0.510932,-0.357811,-0.031369,0.392929,-0.174352,0.237411,-0.929601


In [0]:
# splitting the data in a test and train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [134]:
# Building a linear regression model 
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

# Building a decision tree regression model 
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# Building a random forest regression model 
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators = 200)
forest.fit(X_train, y_train)

# Building Laaso Regularisation regression model 
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [135]:
from sklearn.metrics import mean_squared_error

# Multiple linear regression
y_pred_test = lm.predict(X_test)
y_pred_train = lm.predict(X_train)
rmse_test = round(np.sqrt(mean_squared_error(y_test, y_pred_test)),2)
rmse_train = round(np.sqrt(mean_squared_error(y_train, y_pred_train)),2)

# Decision Tree regression
y_pred_test_tree = tree.predict(X_test)
y_pred_train_tree = tree.predict(X_train)
rmse_test_tree = round(np.sqrt(mean_squared_error(y_test, y_pred_test_tree)),2)
rmse_train_tree = round(np.sqrt(mean_squared_error(y_train, y_pred_train_tree)),2)

# Random forrest regression 
y_pred_test_forest = forest.predict(X_test)
y_pred_train_forest = forest.predict(X_train)
rmse_test_forest = round(np.sqrt(mean_squared_error(y_test, y_pred_test_forest)),2)
rmse_train_forest = round(np.sqrt(mean_squared_error(y_train, y_pred_train_forest)),2)

# Lasso regression
y_pred_lasso_test = lasso.predict(X_test)
y_pred_lasso_train = lasso.predict(X_train)
rmse_test_lasso =  round(np.sqrt(mean_squared_error(y_test, y_pred_lasso_test)),2)
rmse_train_lasso =  round(np.sqrt(mean_squared_error(y_train, y_pred_lasso_train)),2)

data = {'Regression model':  ['Mulitiple linear', 'Decision Tree', 'Random Forest', 'Lasso'],
        'RMSE Training': [rmse_train, rmse_train_tree, rmse_train_forest, rmse_train_lasso],
        'RMSE Testing' : [rmse_test, rmse_test_tree, rmse_test_forest, rmse_test_lasso]
        }

df = pd.DataFrame (data, columns = ['Regression model', 'RMSE Training', 'RMSE Testing'])
df.head()


,Regression model,RMSE Training,RMSE Testing
0,Mulitiple linear,797.58,800.84
1,Decision Tree,0.00,1092.83
2,Random Forest,285.26,768.69
3,Lasso,797.58,800.85


In [137]:
# lests see the coefficients for the lasso regression
coef = pd.DataFrame(lasso.coef_,X.columns, columns = ['coef'])
print(coef)

                        coef
Weekday             4.005102
Pickup - Time       9.466096
Distance (KM)     582.782913
Temperature         0.114003
Pickup Lat         14.830597
Pickup Long       -39.970480
Destination Lat   -13.661193
Destination Long   -7.407806
No_Of_Orders      -65.509160
Age                14.314539
Average_Rating    -30.435230
Waiting time       23.438887


In [138]:
# submission

# Multiple linear regression
y_pred_lin = lm.predict(Test_s)

# Decision Tree regression
y_pred_tree = tree.predict(Test_s)

# Random forrest regression 
y_pred_forest = forest.predict(Test_s)


# Submission
sb_dataframe = Tests.iloc[:,:3]
sb_dataframe['Time from pick up to arrival 1'] = lasso.predict(Test_s)
sb_dataframe.drop('User Id', inplace = True, axis = 1)
sb_dataframe.drop('Vehicle Type', inplace = True, axis = 1)

print(sb_dataframe)
sb_dataframe.to_csv('submission_lasso.csv', index = False)

            Order No  Time from pick up to arrival 1
0     Order_No_19248                     1433.401488
1     Order_No_12736                     1133.704003
2       Order_No_768                     1121.675315
3     Order_No_15332                     1146.140638
4     Order_No_21373                     1030.865713
...              ...                             ...
7063   Order_No_3612                     1103.185634
7064   Order_No_7657                     2722.249291
7065   Order_No_1969                     1670.741439
7066  Order_No_10591                     2647.337311
7067   Order_No_1603                     1487.656665

[7068 rows x 2 columns]
